# Evaluation

In [ ]:
import json
import os

import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

### Summary Length

In [ ]:
directory = "./evaluation/summary/"
# Eventuell die Ergebnisse in einer neuen CSV abspeichern, wo man dann actual und predcition nebeneinander legt?
# summaryDataset = pd.read_csv('data/summary/summaryDataset.csv')

actual = np.array([])
predicted = np.array([])

for file in os.listdir(directory):
    filename = os.fsdecode(file)

    with open(directory + filename) as f:
        result = json.load(f)
    
    actual = np.append(actual, result["violation_actual"])
    predicted = np.append(predicted, result["output"]["violation_predicted"])

In [ ]:
confusion_matrix = metrics.confusion_matrix(actual, predicted, labels=["TRUE", "FALSE"])
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["TRUE", "FALSE"])

In [ ]:
cm_display.plot()
plt.show()

In [ ]:
Accuracy = metrics.accuracy_score(actual, predicted)
Accuracy